In [15]:
from datetime import datetime, time, timedelta
import math
import random
random.seed(datetime.now().timestamp())
import pandas as pd
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt

In [16]:
def random_roll():
    last_time = datetime.now().timestamp()
    while True:
        time_now = datetime.now().timestamp()
        if last_time != time_now:
            random.seed(time_now)
            last_time = time_now
        else:
            random.seed(random.random())
        yield random.random()
rnd = random_roll()

class Model:
    def __init__(self, config):
        self.attributes = ["arrival_time","service_span","service_start","system_wait","queue_span","elimination_time","eliminated","queue_length"]
        self.config = config
        i_accumulator = 0
        for i in range(len(self.config.incoming_distr)):
            i_accumulator += self.config.incoming_distr[i][1]
            self.config.incoming_distr[i].append(i_accumulator)
        self.counter = 0
        self.store = []
        self.queue = []
        self.arrival_time = 0
    def random_gen(self):
        for i in range(len(self.config.incoming_distr)):
            if self.config.incoming_distr[i][2] >= next(rnd):
                self.arrival_time += self.config.incoming_distr[i][0][0] + next(rnd) * self.config.incoming_distr[i][0][1]
                break
        self.service_span = self.config.service_span_from + self.config.service_span_range * next(rnd)
        self.queue_span = self.config.queue_span_from + self.config.queue_span_range * next(rnd)
    def time_increment(self):
        self.counter += 1
        self.random_gen()
        service_start = self.arrival_time + self.queue_span
        system_wait_span = self.service_span + self.queue_span
        eliminated = ""
        eliminated_list = []
        for i in self.queue:
            if i[1] < self.arrival_time:
                eliminated_list.append(i)
                eliminated += f'{i[0]} '
            else:
                break
        for i in eliminated_list:
            self.queue.remove(i)
        elimination_time = self.arrival_time + system_wait_span
        self.queue.append([self.counter, elimination_time])
        return [
            self.arrival_time,
            self.service_span,
            service_start,
            system_wait_span,
            self.queue_span,
            elimination_time,
            eliminated,
            len(self.queue)-1]
    def model(self, timespan = 3650):
        for _ in range(timespan):
            self.store.append(self.time_increment())
        df = pd.DataFrame(self.store, columns=self.attributes)
        print(df)
        return df

class Object:
    def __init__(self, **attributes):
        self.__dict__.update(attributes)

In [17]:
config = Object(
    incoming_distr = [
        [[1,1], 0.05],
        [[2,10], 0.2],
        [[11,20], 0.2],
        [[21,30], 0.3],
        [[31,40], 0.1],
        [[40,60], 0.15]],
    queue_count = 1,
    queue_span_from = 1,
    queue_span_range = 10,
    service_span_from = 1,
    service_span_range = 8,
    start_hour = 8)

model = Model(config)
df = model.model(15)

    arrival_time  service_span  service_start  system_wait  queue_span  elimination_time eliminated  queue_length
0      28.669473      1.171137      35.237453     7.739117    6.567981         36.408590                        0
1      30.127682      6.622553      33.311287     9.806158    3.183605         39.933840                        1
2      36.209071      4.683201      40.149939     8.624068    3.940868         44.833140                        2
3      42.478020      5.486449      50.527978    13.536406    8.049958         56.014427       1 2              1
4      57.654564      8.859019      62.554733    13.759189    4.900169         71.413753       3 4              0
5      87.042293      6.035690      95.093646    14.087043    8.051354        101.129336         5              0
6     112.296846      4.250635     121.108316    13.062105    8.811470        125.358951         6              0
7     140.697582      2.334706     151.681741    13.318865   10.984159        154.016446

In [18]:
df["arrival_time"].diff()

0           NaN
1      1.458210
2      6.081389
3      6.268949
4     15.176544
5     29.387729
6     25.254554
7     28.400735
8      7.166604
9     22.258027
10    24.210332
11    15.617844
12    43.387862
13    11.209219
14    12.318148
Name: arrival_time, dtype: float64

In [19]:
def to_time(minutes, start=datetime(2024, 3, 30, 8)):
    timestamp_minutes = math.floor(minutes)
    timestamp_seconds = math.floor((minutes - timestamp_minutes) * 60)
    timestamp = start + timedelta(minutes=timestamp_minutes, seconds=timestamp_seconds)
    return timestamp

time_update = lambda x: to_time(x).strftime('%H:%M:%S')
df["arrival_time"] = df["arrival_time"].apply(time_update)
df["service_start"] = df["service_start"].apply(time_update)
df["elimination_time"] = df["elimination_time"].apply(time_update)

In [13]:
print(df)

   arrival_time  service_span service_start  system_wait  queue_span elimination_time eliminated  queue_length
0      08:25:51      3.049780      08:28:19     5.514122    2.464342         08:31:22                        0
1      09:30:40      5.921409      09:31:45     7.000048    1.078639         09:37:40         1              0
2      10:01:19      5.326318      10:06:43    10.731274    5.404957         10:12:03         2              0
3      10:23:11      1.436206      10:27:38     5.886641    4.450435         10:29:04         3              0
4      10:48:08      5.138417      10:49:26     6.439481    1.301064         10:54:35         4              0
5      10:49:15      6.623766      10:58:07    15.485128    8.861361         11:04:44                        1
6      11:20:42      6.353883      11:27:15    12.908337    6.554455         11:33:36       5 6              0
7      11:30:23      3.730776      11:34:48     8.147900    4.417125         11:38:32                        1
8